<a href="https://colab.research.google.com/github/Siddhantg2002/Basic-Concepts-3/blob/main/Preprocessing_Categorical_columns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
x_train_full= pd.read_csv('train.csv',index_col='Id')
x_test_full=pd.read_csv('test.csv',index_col='Id')

x_train_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = x_train_full.SalePrice
x_train_full.drop(['SalePrice'], axis=1, inplace=True)
 
#Here we first dropping the columns then splitting the data set whereas above we first spitted the dataset then dropped the columns
cols_with_missing = [col for col in x_train_full.columns if x_train_full[col].isnull().any()] 
x_train_full.drop(cols_with_missing, axis=1, inplace=True)
x_test_full.drop(cols_with_missing, axis=1, inplace=True)

x_train, x_valid, y_train, y_valid = train_test_split(x_train_full,y,train_size=0.8, test_size=0.2,random_state=0)
x_train.head()

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
619,20,RL,11694,Pave,Reg,Lvl,AllPub,Inside,Gtl,NridgHt,...,108,0,0,260,0,0,7,2007,New,Partial
871,20,RL,6600,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,...,0,0,0,0,0,0,8,2009,WD,Normal
93,30,RL,13360,Pave,IR1,HLS,AllPub,Inside,Gtl,Crawfor,...,0,44,0,0,0,0,8,2009,WD,Normal
818,20,RL,13265,Pave,IR1,Lvl,AllPub,CulDSac,Gtl,Mitchel,...,59,0,0,0,0,0,7,2008,WD,Normal
303,20,RL,13704,Pave,IR1,Lvl,AllPub,Corner,Gtl,CollgCr,...,81,0,0,0,0,0,1,2006,WD,Normal


In [ ]:
#Function to compare diffrent approaches
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
def accuracy_score(x_train,x_val,y_train,y_val):
    model=RandomForestRegressor(n_estimators=500,random_state=0)
    model.fit(x_train,y_train)
    pred=model.predict(x_val)
    return mean_absolute_error(y_val,pred)

In [ ]:
#DROPPING THE CATEGORICAL DATA COLUMNS
drop_x_train=x_train.select_dtypes(exclude=['object'])
drop_x_valid=x_valid.select_dtypes(exclude=['object'])
print("MAE from Approach 1 (Drop categorical variables):")
print(accuracy_score(drop_x_train, drop_x_valid, y_train, y_valid))

MAE from Approach 1 (Drop categorical variables):
17466.586351598173


#ORDINAL CODING


In [ ]:

# Categorical columns in the training data
object_cols= [col for col in x_train.columns if x_train[col].dtype=='object']

# Columns that can be safely ordinal encoded
safely_ordinated_cols=[col for col in object_cols if set(x_valid[col]).issubset(set(x_train[col]))]

# Problematic columns that will be dropped from the dataset
Problematic_cols=list(set(object_cols)-set(safely_ordinated_cols))

print('Categorical columns that will be ordinal encoded:', safely_ordinated_cols)
print('\nCategorical columns that will be dropped from the dataset:', Problematic_cols)

Categorical columns that will be ordinal encoded: ['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'BldgType', 'HouseStyle', 'RoofStyle', 'Exterior1st', 'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation', 'Heating', 'HeatingQC', 'CentralAir', 'KitchenQual', 'PavedDrive', 'SaleType', 'SaleCondition']

Categorical columns that will be dropped from the dataset: ['Condition2', 'RoofMatl', 'Functional']


In [ ]:
from sklearn.preprocessing import OrdinalEncoder
# Dropping categorical columns that will not be encoded
label_x_train=x_train.drop(Problematic_cols,axis=1)
label_x_valid=x_valid.drop(Problematic_cols,axis=1)

ordinal_encoder=OrdinalEncoder()
label_x_train[safely_ordinated_cols]=ordinal_encoder.fit_transform(x_train[safely_ordinated_cols])
label_x_valid[safely_ordinated_cols]=ordinal_encoder.transform(x_valid[safely_ordinated_cols])
print("MAE from Approach 2 (Ordinal Encoding):") 
print(accuracy_score(label_x_train, label_x_valid, y_train, y_valid))

MAE from Approach 2 (Ordinal Encoding):
16787.371783105023


# ONE HOT CODE ENCODING

In [ ]:
#Checking cardinality
# Get number of unique entries in each column with categorical data
object_nunique = list(map(lambda col: x_train[col].nunique(), object_cols))
d = dict(zip(object_cols, object_nunique))

# Print number of unique entries by column, in ascending order
sorted(d.items(), key=lambda x: x[1])

[('Street', 2),
 ('Utilities', 2),
 ('CentralAir', 2),
 ('LandSlope', 3),
 ('PavedDrive', 3),
 ('LotShape', 4),
 ('LandContour', 4),
 ('ExterQual', 4),
 ('KitchenQual', 4),
 ('MSZoning', 5),
 ('LotConfig', 5),
 ('BldgType', 5),
 ('ExterCond', 5),
 ('HeatingQC', 5),
 ('Condition2', 6),
 ('RoofStyle', 6),
 ('Foundation', 6),
 ('Heating', 6),
 ('Functional', 6),
 ('SaleCondition', 6),
 ('RoofMatl', 7),
 ('HouseStyle', 8),
 ('Condition1', 9),
 ('SaleType', 9),
 ('Exterior1st', 15),
 ('Exterior2nd', 16),
 ('Neighborhood', 25)]

In [ ]:
#We will only encode those columns whose cardinality is less or equal to 15 else will be ignored.
low_cardinality_cols=[col for col in object_cols if x_train[col].nunique()<=15]
high_cardinality_cols = list(set(object_cols)-set(low_cardinality_cols))
print('Categorical columns that will be one-hot encoded:', low_cardinality_cols)
print('\nCategorical columns that will be dropped from the dataset:', high_cardinality_cols)

Categorical columns that will be one-hot encoded: ['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'ExterQual', 'ExterCond', 'Foundation', 'Heating', 'HeatingQC', 'CentralAir', 'KitchenQual', 'Functional', 'PavedDrive', 'SaleType', 'SaleCondition']

Categorical columns that will be dropped from the dataset: ['Neighborhood', 'Exterior2nd']


In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(x_train[low_cardinality_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(x_valid[low_cardinality_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = x_train.index
OH_cols_valid.index = x_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_x_train = x_train.drop(object_cols, axis=1)
num_x_valid = x_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_x_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_x_valid, OH_cols_valid], axis=1)


/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
